In [3]:
! pip install elasticsearch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 kB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 3.4 MB/s eta 0:00:00


In [3]:
try:
  import os
  import elasticsearch
  from elasticsearch import Elasticsearch
  import numpy as np
  import pandas as pd
  import sys
  import json
  from ast import literal_eval
  from tqdm import tqdm
  import datetime
  from elasticsearch import helpers
  

except Exception as e:
  print(f"error: {e}")

In [1]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

user        6421    6418  0 14:17 ?        00:00:00 grep elasticsearch


In [4]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200,'scheme':'http'}])
# Check if python is connected to elasticsearch
print(es.ping())

True


In [5]:
# Elasticsearch 인덱스 생성
index_name = 'txt_data_index'
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

In [26]:
! pip install sentence-transformers
! pip install langchain unstructured openai chromadb Cython tiktoken pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00a 0:00:01
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-n

In [6]:
from langchain.text_splitter import CharacterTextSplitter

In [33]:
! pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

#OAuth 2.0 클라이언트 ID에서 다운로드한 JSON 파일 경로
CLIENT_SECRET_FILE = '/home/user/Downloads/<path>'
API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

# 인증 흐름 설정
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)

# 사용자 인증 및 토큰 획득
credentials = flow.run_local_server(port=0)

# API 빌드
service = build(API_NAME, API_VERSION, credentials=credentials)

# 특정 폴더 ID
folder_id_school = '<folder_id>'   # school_text
folder_id_project = '<folder_id>'   # project_data
folder_id_txt = '<folder_id>'   # txt

# 여러 개의 폴더 ID를 포함한 리스트
folder_ids = [folder_id_school, folder_id_project, folder_id_txt]

# 각 폴더 ID를 사용하여 OR 연산자를 적용한 조건 생성
q_condition = ' or '.join([f"'{folder_id}' in parents" for folder_id in folder_ids])


# pageToken을 사용하여 페이지를 나누어 결과를 받아오기
page_token = None
files = []

while True:
    results = service.files().list(
        q=q_condition,
        fields="nextPageToken, files(id, name)",
        pageToken=page_token
    ).execute()

    files.extend(results.get('files', []))
    page_token = results.get('nextPageToken')

    if not page_token:
        break

In [73]:
# text_splitter 초기화
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)

i = 1

for file in tqdm(files, desc="파일 처리 중", unit="개"):
    file_id = file['id']
    file_name = file['name']
    content = service.files().get_media(fileId=file_id).execute()
    data = content.decode('utf-8')

    # text_splitter를 사용하여 텍스트 분리
    chunks = text_splitter.split_text(data)

    # 데이터를 Elasticsearch에 색인화
    for j, chunk in enumerate(chunks):
        document = {
            'text': chunk.strip(),  # 각 줄의 텍스트를 'text' 필드에 저장
            'file_name': file_name  # 파일 이름도 저장 (옵션)
        }
        es.index(index=index_name, id=i * len(file_name) + j + 1, body=document)
    i += 1


파일 처리 중: 100%|██████████| 1298/1298 [17:44<00:00,  1.22개/s]


In [74]:
# 색인화가 완료되었다면 몇 초 기다려서 Elasticsearch에 적용되도록 합시다.
import time
time.sleep(5)

In [78]:
# 특정 쿼리로 검색하기
search_query = '학생지원팀 전화번호?' # 여기 입력
search_body = {
    'query': {
        'match': {
            'text': search_query
        }
    },
    'size': 5 # 최대 5개의 문서만 반환(토큰 4000제한 때문에)
}

In [79]:
# 검색 실행
results = es.search(index=index_name, body=search_body)

In [80]:
# 결과 출력
print(f"검색어 '{search_query}'에 대한 검색 결과:")
for hit in results['hits']['hits']:
    print(hit['_source']['text'])

검색어 '학생지원팀 전화번호?'에 대한 검색 결과:
2023학년도 KIT역량장학금
K-WEGO 장학제도 안내
문의
마일리지 관련
학생역량개발팀 478-7281~3
취업진로상담 교과목 관련
대학일자리플러스센터 478-7984
선발 및 지급 관련
학생지원팀 478-7045
학생성공처 -
https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=429943&article.offset=260&articleLimit=10에서 가져온 내용:
2023년 푸른등대 삼성기부장학금 신규장학생 신청 안내
1.
장학금명
:
푸른등대 삼성기부장학금
(
삼성 기부금 재원 장학사업
)
2.
지원유형 및 배정인원
-
사회적 배려계층 생활비 지원 유형
: 3
명
3.
유형별 장학생 선발 및 장학금 지원 내용
사업유형
사회적 배려계층 생활비 지원 유형
선발인원
•
3
명
(
사회적 배려계층 생활비 지원 유형
)
선발대상
(
신청자격
)
•
한국장학재단에서
2023
년
1
학기
학자금지원
3
구간 이하
로 확인 가능한
사회적
배려계층 가정의 대학생
(
장애인 가정
,
새터민
(
탈북주민
)
가정
,
가정외보호 시설 출신자
,
학생가장 본인
,
조손
가정 자녀
,
기초생활수급자
·
차상위계층 가정의 본인 또는 자녀
,
｢
한부모가
족
지원법
｣
에
따라 한부모가족 증명서 발급 가능한 자
,
다문화가정
,
주거비 지원
필요자
,
기타 사회적배려계층
)
※
상세자격(
첨부파일. [
붙임
2~4.]
장학생 자격 및 제출서류
참조)
•
직전학기 성적
3.3(87
점
)
이상 재학생
•
2023
년
1
학기 기준
,
정규 잔여학기가
2
개 학기 이상인 학생
추천제외
(
신청불가
)
•
2023
년
1
학기 푸른등대 기부장학금 장학생
•
2023
년
1
학기 생활비 및 수학보조비 성격 장학금
100
만원 이상 수혜자
•
신입생 및 편입생
장학생 추천
(
선발
)
기준
•
(1
순위
)
학자금 지원구간이 낮은 자
→
(2
순위
)
직전학기 평균평점이